In [ ]:
import cv2
import numpy as np
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
import queue
import threading
import cv2
import ipywidgets as widgets
KMEANSVALUE = 1
sp_spatial = 1
sp_range = 1
num_segments = 1
buffer = queue.Queue()

labelTest = widgets.Label(value='Test')
nosignal = cv2.imread('no_signal.jpg')
nosignal = cv2.resize(nosignal, (320,240))


style = {'description_width': 'initial'}


morphologyExIteratorSlider = widgets.IntSlider(min=0, max=7, step=1, value=3)
morphologyExIterator = 3

dilateIteratorSlider = widgets.IntSlider(min=0, max=7, step=1, value=3)
dilateIterator = 3


kmeans_status = False
mean_shift_status = False



adaptive_threshold_map = {'ADAPTIVE_THRESH_MEAN_C': cv2.ADAPTIVE_THRESH_MEAN_C, 'ADAPTIVE_THRESH_GAUSSIAN_C': cv2.ADAPTIVE_THRESH_GAUSSIAN_C}
adaptive_threshold_typeDropDown = widgets.Dropdown(
    options=list(adaptive_threshold_map.keys()),
    description='Adaptive Threshold Type:',
    style=style
    )

threshold_map = {'THRESH_BINARY': cv2.THRESH_BINARY, 'THRESH_BINARY_INV': cv2.THRESH_BINARY_INV, 'THRESH_TRUNC': cv2.THRESH_TRUNC, 'THRESH_TOZERO': cv2.THRESH_TOZERO, 'THRESH_TOZERO_INV': cv2.THRESH_TOZERO_INV}
threshold_typeDropdown = widgets.Dropdown(
    options=list(threshold_map.keys()),
    description='Threshold Type:',
    style=style
    )

# Create global variables for adjustable parameters
adaptive_threshold_type = cv2.ADAPTIVE_THRESH_MEAN_C
threshold_type = cv2.THRESH_BINARY
block_size = widgets.IntSlider(value=11, min=3, max=21, step=2, description='Block Size:')
constant = widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description='Constant:')


maXSlider = widgets.IntSlider(value=255, min=0, max=255, description='Maksimum:')
constantVariable = 2
blockSizeVariable = 11
maxVariable = 255





image_widget = widgets.Image(format='jpeg')
image_widget_mean_shift = widgets.Image(format='jpeg')


image_widget_threshold = widgets.Image(format='jpeg')
image_widget_watershed = widgets.Image(format='jpeg')

slider_kmeans_value = widgets.IntSlider(min=1, max=100, step=1, value=1)
slider_sp_spatial = widgets.IntSlider(min=1, max=8, step=1, value=sp_spatial)
slider_sp_range = widgets.IntSlider(min=1, max=8, step=1, value=sp_range)
slider_num_segments = widgets.IntSlider(min=1, max=8, step=1, value=num_segments)


text_box_fps_kmeans = widgets.Text(value='0', disabled=True)
text_box_fps_kmeans.layout.width = '96%'
text_box_kvalue_kmeans = widgets.Text(value='0', disabled=True)
text_box_kvalue_kmeans.layout.width = '96%'

text_box_fps_mean_shift = widgets.Text(value='0', disabled=True)
text_sp_spatial = widgets.Text(value='0', disabled=True)
text_sp_range = widgets.Text(value='0', disabled=True)
text_num_segments = widgets.Text(value='0', disabled=True)





global_frame = np.zeros((320,240,3), np.uint8)




threshold_type 0
threshold_type 0
threshold_type 1
threshold_type 1
threshold_type 1
adaptive_threshold_type 0
adaptive_threshold_type 0
adaptive_threshold_type 1
adaptive_threshold_type 1
adaptive_threshold_type 1
constant FloatSlider(value=2.0, description='Constant:', max=10.0)
constant FloatSlider(value=1.9, description='Constant:', max=10.0)
constant FloatSlider(value=1.9, description='Constant:', max=10.0)
constant FloatSlider(value=1.9, description='Constant:', max=10.0)
constant FloatSlider(value=1.8, description='Constant:', max=10.0)
constant FloatSlider(value=1.8, description='Constant:', max=10.0)
constant FloatSlider(value=1.8, description='Constant:', max=10.0)
constant FloatSlider(value=1.7, description='Constant:', max=10.0)
constant FloatSlider(value=1.7, description='Constant:', max=10.0)
constant FloatSlider(value=1.7, description='Constant:', max=10.0)
constant FloatSlider(value=1.6, description='Constant:', max=10.0)
constant FloatSlider(value=1.6, description='Con

In [9]:
def mean_shift_segmentation(imageMean):
    
    global sp_spatial
    global sp_range
    global num_segments
    hsv = cv2.cvtColor(imageMean, cv2.COLOR_BGR2HSV)
    dst = hsv.copy()
    segmented_image = cv2.pyrMeanShiftFiltering(hsv ,sp_spatial, sp_range, dst, num_segments)
    segmented_image = cv2.cvtColor(segmented_image, cv2.COLOR_HSV2BGR)
    image_widget_mean_shift.value = cv2.imencode('.jpg', dst)[1].tobytes()


In [10]:
def segmen(imageSegmen):
    global KMEANSVALUE

    before_time = time.time()       
    Z = imageSegmen.reshape((-1,3))
    Z = np.float32(Z)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center = cv2.kmeans(Z, KMEANSVALUE, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((imageSegmen.shape))

    image_widget.value = cv2.imencode('.jpg', res2)[1].tobytes()


In [11]:
def threshHolding(imageThresh):

    global maxVariable
    global adaptive_threshold_type
    global threshold_type
    global blockSizeVariable
    global constantVariable
    

    gray_image = cv2.cvtColor(imageThresh, cv2.COLOR_BGR2GRAY)
    # Apply thresholding
    # (1) Simple Thresholding
    thresh, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

    # (2) Otsu's Thresholding (automatic threshold selection)
    ret, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # (3) Adaptive Thresholding (local thresholding)
    binary_image = cv2.adaptiveThreshold(gray_image, maxValue=maxVariable,
                                     adaptiveMethod=adaptive_threshold_type,
                                     thresholdType=threshold_type,
                                     blockSize=blockSizeVariable,
                                     C=constantVariable)

    image_widget_threshold.value = cv2.imencode('.jpg', binary_image)[1].tobytes()
    
    # create  sliders and dropdowns for the thresholding
    

In [12]:
def watershed(imageWater):
    global MorphTypesListVar
    global morphologyExIterator
    global dilateIterator
    global distanceTypeVar
    global maskSize
    global thresh_constantsVar1
    global thresh_constantsVar2
    

    gray = cv2.cvtColor(imageWater,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    
    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    # cv2.imshow("name", markers)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    markers = cv2.watershed(imageWater,markers)
    imageWater[markers == -1] = [255,0,0]
    image_widget_watershed.value = cv2.imencode('.jpg', imageWater)[1].tobytes()

In [13]:
def update_global_variable(change):
    global KMEANSVALUE
    global image
    KMEANSVALUE = slider_kmeans_value.value
    segmen(image)
slider_kmeans_value.observe(update_global_variable)

def update_global_variable_sp_spatial(change):
    global image
    global sp_spatial
    sp_spatial = slider_sp_spatial.value
    mean_shift_segmentation(image)
slider_sp_spatial.observe(update_global_variable_sp_spatial)

def update_global_variable_sp_range(change):
    global sp_range
    global image
    sp_range = slider_sp_range.value
    mean_shift_segmentation(image)
slider_sp_range.observe(update_global_variable_sp_range)

def update_global_variable_num_segments(change):
    global num_segments
    global image
    num_segments = slider_num_segments.value
    mean_shift_segmentation(image)
slider_num_segments.observe(update_global_variable_num_segments)


def update_global_variable_block_size(change):
    global block_size
    global blockSizeVariable
    global image
    blockSizeVariable = block_size.value

    labelTest.value = str(blockSizeVariable)
    threshHolding(image)
# create observer for the block_size slider
block_size.observe(update_global_variable_block_size)

def update_global_variable_constant(change):
    global constantVariable
    global image
    constantVariable = constant.value
   
    labelTest.value = str(constantVariable)
    threshHolding(image)

# create observer for the constant slider
constant.observe(update_global_variable_constant)

def update_global_variable_threshold_type(change):

    global threshold_type
    global image
    threshold_type = threshold_map[threshold_typeDropdown.value]

    labelTest.value = str(threshold_type)
    threshHolding(image)
threshold_typeDropdown.observe(update_global_variable_threshold_type)

def update_global_variable_adaptive_threshold_type(change):

    global adaptive_threshold_type
    global image
    adaptive_threshold_type = adaptive_threshold_map[adaptive_threshold_typeDropDown.value]

    labelTest.value = str(adaptive_threshold_type)
    threshHolding(image)
adaptive_threshold_typeDropDown.observe(update_global_variable_adaptive_threshold_type)


In [14]:
# kmeans components
label_kmeans_value = widgets.Label(value=str('K VALUE (K-MEANS)'))
label_kmeans_value.layout.width = '100%'

vertical_box_kmeans = widgets.VBox([label_kmeans_value, slider_kmeans_value, image_widget])
# display(vertical_box_kmeans)

# mean shift components
label_sp_spatial = widgets.Label(value=str('SPATIAL WINDOW SIZE'))
label_sp_range = widgets.Label(value=str('RANGE WINDOW SIZE'))
label_num_segments = widgets.Label(value=str('NUMBER OF SEGMENTS'))


vertical_mean_shift = widgets.VBox([label_sp_spatial, slider_sp_spatial, label_sp_range, slider_sp_range, label_num_segments, slider_num_segments, text_box_fps_mean_shift])
vertical_mean_shift.layout.width = '60%'
horizontal_mean_shift = widgets.HBox([image_widget_mean_shift, vertical_mean_shift])



vertical_kmeans = widgets.VBox([label_kmeans_value, slider_kmeans_value])
vertical_kmeans.layout.width = '60%'
label1 = widgets.Label(value='KMEANS')
label2 = widgets.Label(value='MEAN SHIFT')
label2 = widgets.Label(value='Thresholding')
label3 = widgets.Label(value='Watershed')

horizontal_kmeans = widgets.HBox([image_widget, vertical_kmeans])
# vertical_box_meanshift.layout.width = '60%'


# thresholding components 
label_block_size = widgets.Label(value='Block Size:')
label_constant = widgets.Label(value='Constant:')
label_adaptive_threshold_type = widgets.Label(value='Adaptive Threshold Type:')
label_threshold_type = widgets.Label(value='Threshold Type:')
label_adaptive_threshold = widgets.Label(value='Adaptive Thresholding')
label_threshold = widgets.Label(value='Thresholding')

vertical_Slider = widgets.VBox([label_block_size, block_size, label_constant, constant])
# vertical_Slider.layout.width = '60%'

vertical_threshold = widgets.VBox([threshold_typeDropdown, adaptive_threshold_typeDropDown])

vertical_all = widgets.VBox([vertical_Slider, vertical_threshold])

horizontal_threshold = widgets.HBox([image_widget_threshold, vertical_all])
image_widget_threshold.layout.width = '40%'
image = cv2.imread('no_signal.jpg')
image = cv2.resize(image, (320,240))
threshHolding(image)
# display(horizontal_threshold)
display(labelTest)

# watershed components
label_morphologyExIterator = widgets.Label(value='MorphologyEx Iterator:')
label_dilateIterator = widgets.Label(value='Dilate Iterator:')
label_distanceType = widgets.Label(value='Distance Type:')
label_maskSize = widgets.Label(value='Mask Size:')
label_thresh_constantsVar1 = widgets.Label(value='Thresh Constants Var1:')
label_thresh_constantsVar2 = widgets.Label(value='Thresh Constants Var2:')
label_MorphTypesListVar = widgets.Label(value='Morph Types List Var:')

MorphTypesListVar = cv2.MORPH_OPEN
distanceTypeVar = cv2.DIST_L2
maskSize = 5
thresh_constantsVar1 = cv2.THRESH_BINARY
thresh_constantsVar2 = cv2.THRESH_BINARY_INV








segmen(image)
mean_shift_segmentation(image)
water_image = image.copy()
watershed(water_image)
threshHolding(image)

# # display(mainBox)

# image_widget.layout.width = '40%'
# image_widget_mean_shift.layout.width = '40%'
# image_widget_threshold.layout.width = '40%'
# image_widget_watershed.layout.width = '40%'



grid = widgets.GridspecLayout(2, 2)
grid[0, 0] = horizontal_kmeans
grid[0, 1] = horizontal_mean_shift
grid[1, 0] = image_widget_watershed
grid[1, 1] = horizontal_threshold
display(grid)




Label(value='Test')

GridspecLayout(children=(HBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00…